In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

HAGO ENDPINT CON FECHAS + FILTRADO

In [2]:
# Definir el rango de fechas que te interesa
fecha_inicio = "2025-03-01"
fecha_fin = "2025-03-02"
url = f"https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json?dtstart={fecha_inicio}&dtend={fecha_fin}"


In [3]:
# Realizar la solicitud
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()
    
    # Convertir las fechas de inicio y fin a objetos datetime
    fecha_inicio_dt = datetime.strptime(fecha_inicio, "%Y-%m-%d")   # ESTO NO LO ENTIENDO
    fecha_fin_dt = datetime.strptime(fecha_fin, "%Y-%m-%d")
    
    # Filtrar los datos para asegurarse de que solo incluyan eventos dentro del rango de fechas
    eventos_filtrados = []
    for evento in data['@graph']:
        fecha_evento = datetime.strptime(evento['dtstart'], "%Y-%m-%d %H:%M:%S.%f")
        if fecha_inicio_dt <= fecha_evento <= fecha_fin_dt:
            eventos_filtrados.append(evento)
    
    # Mostrar los datos filtrados
    print(eventos_filtrados)
else:
    print("Error al obtener los datos:", response.status_code)

[{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/evento/12682299-asalto-lectura-tres-cerditos-mas.json', '@type': 'https://datos.madrid.es/egob/kos/actividades/RecitalesPresentacionesActosLiterarios', 'id': '12682299', 'title': 'Asalto a la lectura. Tres cerditos y más', 'description': '', 'free': 1, 'price': '', 'dtstart': '2025-03-01 11:00:00.0', 'dtend': '2025-03-01 23:59:00.0', 'time': '11:00', 'excluded-days': '', 'audience': 'Niños,Familias', 'uid': '12682299', 'link': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=6aec3e3332fe4910VgnVCM1000001d4a900aRCRD', 'event-location': 'Centro Cultural Clara del Rey - Museo ABC (Centro)', 'references': {'@id': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=1ad0d88f2de54910VgnVCM1000001d4a900aRCRD'}, 'relation': {'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/11876732-centro-cultural-clara-rey-museo

In [4]:
data_dic = eventos_filtrados

In [5]:
eventos = []

for evento in data_dic:
    id_evento = evento.get("id", None)
    nombre_evento = evento.get("title", "")
    url_evento = evento.get("link", "")
    horario = evento.get("time", None)
    organizacion = evento.get("organization", {}).get("organization-name", None)

    # 🔍 1. Intentar obtener la dirección desde "event-location"
    direccion = evento.get("event-location", None)

    # 🔍 2. Si no hay dirección, buscar en "description"
    if not direccion:
        descripcion = evento.get("description", "")
        if "Lugar:" in descripcion:  # ✅ Verifica si la descripción contiene "Lugar:"
            direccion = descripcion.split("Lugar:")[1].strip()  # Extrae la parte después de "Lugar:"
    
    # 🔍 3. Obtener código postal
    direccion_info = evento.get("address", {}).get("area", {})
    codigo_postal = direccion_info.get("postal-code", None)

    # Agregar a la lista de eventos
    eventos.append((id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, evento.get("dtstart", ""), evento.get("dtend", ""), organizacion))

# Convertir a DataFrame
df_api = pd.DataFrame(eventos, columns=["id_evento", "nombre_evento", "url_evento", "codigo_postal", "direccion", "horario", "fecha_inicio", "fecha_fin", "organizacion"])


In [6]:
shape =df_api.shape
print(shape)
df_api.sample(10)

(35, 9)


,id_evento,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion
5,12691752,Carnaval en Matadero,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,Matadero Madrid,10:30,2025-03-01 10:30:00.0,2025-03-01 23:59:00.0,Matadero Madrid
25,12690327,Senderismo urbano. Emilia Pardo Bazán: mujer e...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,Centro de Información y Educación Ambiental de...,11:00,2025-03-01 11:00:00.0,2025-03-01 23:59:00.0,Centro de Información y Educación Ambiental de...
3,12682149,Baile de Carnaval. Concierto de Versioneta,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,plaza de Puerto Rubio (antigua plaza Vieja). V...,21:15,2025-03-01 21:15:00.0,2025-03-01 23:59:00.0,None
10,12039151,De Casa de Campo a Madrid Río I,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28011,Centro de Información y Educación Ambiental de...,11:00,2025-03-01 11:00:00.0,2025-03-01 23:59:00.0,Centro de Información y Educación Ambiental de...
15,12233828,Itinerario Ornitológico Parque Emperatriz Marí...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,09:00,2025-03-01 09:00:00.0,2025-03-01 23:59:00.0,None
30,12686923,Talleres Infantiles de Carnaval. Centros Socio...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,11:00,2025-03-01 11:00:00.0,2025-03-01 23:59:00.0,None
9,12690609,Cuatro historias de alquiler,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,Centro Cultural Buero Vallejo (San Blas - Cani...,19:00,2025-03-01 19:00:00.0,2025-03-01 23:59:00.0,Centro Cultural Buero Vallejo (San Blas - Cani...
16,12693346,"Julieta, pero no mucho",http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,Centro Cultural Ciudad Pegaso (San Blas - Cani...,19:00,2025-03-01 19:00:00.0,2025-03-01 23:59:00.0,Centro Cultural Ciudad Pegaso (San Blas - Cani...
24,12694038,Senda Descubre la vida silvestre del parque de...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28026,Centro de Educación Ambiental y Cultural Maris...,10:00,2025-03-01 10:00:00.0,2025-03-01 23:59:00.0,Centro de Educación Ambiental y Cultural Maris...
0,12682299,Asalto a la lectura. Tres cerditos y más,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28015,Centro Cultural Clara del Rey - Museo ABC (Cen...,11:00,2025-03-01 11:00:00.0,2025-03-01 23:59:00.0,Centro Cultural Clara del Rey - Museo ABC (Cen...


In [7]:
def completar_horario(df):
    """
    Extrae la hora de la columna 'fecha_inicio' y la usa para llenar 'horario' si está vacío.
    
    Parámetros:
        df (pd.DataFrame): DataFrame con las columnas 'fecha_inicio' y 'horario'.
        
    Retorna:
        pd.DataFrame: DataFrame con 'horario' corregido.
    """
    # Extraer la hora de 'fecha_inicio' (primeros 16 caracteres para evitar milisegundos)
    df["hora_extraida"] = df["fecha_inicio"].astype(str).str[11:16]  # Formato HH:MM
    
    # Si 'horario' está vacío, completar con 'hora_extraida'
    df["horario"] = df["horario"].mask(df["horario"] == "", df["hora_extraida"])
    
    # Eliminar la columna auxiliar
    df.drop(columns=["hora_extraida"], inplace=True)
    
    return df


In [8]:
completar_horario(df_api)
df_api.sample(10)

,id_evento,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion
31,12693997,Tejiendo Colores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28026,Centro de Educación Ambiental y Cultural Maris...,00:00,2025-03-01 00:00:00.0,2025-03-31 23:59:00.0,Centro de Educación Ambiental y Cultural Maris...
20,12690099,Murdle,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,Matadero Madrid,17:00,2025-03-01 17:00:00.0,2025-03-01 23:59:00.0,Matadero Madrid
6,12693374,Chup-Chup,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,Centro Cultural José Luis López Vázquez (San B...,19:00,2025-03-01 19:00:00.0,2025-03-01 23:59:00.0,Centro Cultural José Luis López Vázquez (San B...
7,12695974,Concurso de Disfraces y Baile con Orquesta,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28041,Centro Municipal de Mayores Nuestra Señora de ...,21:30,2025-03-01 21:30:00.0,2025-03-01 23:59:00.0,Centro Municipal de Mayores Nuestra Señora de ...
32,12686168,"Villaverde Alto. Actuación Infantil, Concurso ...",http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,19:30,2025-03-01 19:30:00.0,2025-03-01 23:59:00.0,None
29,12696013,Talleres de máscaras,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28020,Centro Sociocultural Tetuán,00:00,2025-03-01 00:00:00.0,2025-03-01 23:59:00.0,Centro Sociocultural Tetuán
10,12039151,De Casa de Campo a Madrid Río I,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28011,Centro de Información y Educación Ambiental de...,11:00,2025-03-01 11:00:00.0,2025-03-01 23:59:00.0,Centro de Información y Educación Ambiental de...
1,12691299,Atlantiques + Dahomey,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,Cineteca Madrid,19:30,2025-03-01 19:30:00.0,2025-03-01 23:59:00.0,Cineteca Madrid
27,12696746,Taller de Bancal lasaña en el Huerto Urbano Co...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,11:00,2025-03-01 11:00:00.0,2025-03-01 23:59:00.0,None
24,12694038,Senda Descubre la vida silvestre del parque de...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28026,Centro de Educación Ambiental y Cultural Maris...,10:00,2025-03-01 10:00:00.0,2025-03-01 23:59:00.0,Centro de Educación Ambiental y Cultural Maris...


In [9]:
df_api["fecha_inicio"].value_counts().sort_index()

fecha_inicio
2025-03-01 00:00:00.0    6
2025-03-01 09:00:00.0    1
2025-03-01 10:00:00.0    1
2025-03-01 10:30:00.0    2
2025-03-01 11:00:00.0    7
2025-03-01 12:00:00.0    1
2025-03-01 12:30:00.0    1
2025-03-01 17:00:00.0    1
2025-03-01 17:30:00.0    1
2025-03-01 18:00:00.0    4
2025-03-01 19:00:00.0    3
2025-03-01 19:30:00.0    3
2025-03-01 21:15:00.0    1
2025-03-01 21:30:00.0    3
Name: count, dtype: int64

In [10]:
celdas_vacias = df_api.isnull().sum() + (df_api == '').sum()
celdas_vacias

id_evento         0
nombre_evento     0
url_evento        0
codigo_postal    14
direccion        11
horario           0
fecha_inicio      0
fecha_fin         0
organizacion     13
dtype: int64

## Normalización

In [11]:
def normalizar_fechas(df):
    """
    Elimina las horas de las columnas 'fecha_inicio' y 'fecha_fin', dejando solo 'YYYY-MM-DD'.
    
    Parámetros:
        df (pd.DataFrame): DataFrame con las columnas 'fecha_inicio' y 'fecha_fin'.
        
    Retorna:
        pd.DataFrame: DataFrame con las fechas normalizadas.
    """
    df["fecha_inicio"] = df["fecha_inicio"].astype(str).str[:10]  # Extraer solo YYYY-MM-DD
    df["fecha_fin"] = df["fecha_fin"].astype(str).str[:10]  # Extraer solo YYYY-MM-DD
    
    return df


In [12]:
df_api = normalizar_fechas(df_api)
df_api.sample(3)

,id_evento,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion
6,12693374,Chup-Chup,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,Centro Cultural José Luis López Vázquez (San B...,19:00,2025-03-01,2025-03-01,Centro Cultural José Luis López Vázquez (San B...
12,12694176,Exposición de Pintura `Paisajes´ y `Rincones d...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28021,Centro Sociocultural Marconi (Villaverde),00:00,2025-03-01,2025-03-31,Centro Sociocultural Marconi (Villaverde)
17,12234105,Los Secretos del Capricho: público familiar,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,,10:30,2025-03-01,2025-03-01,None


## Carga de datos

## Paso 1. Conectar con Postgres

In [16]:
import psycopg2
from psycopg2 import OperationalError

def create_connection():
    try:
        # Configurar la conexión
        conn = psycopg2.connect(
            dbname="project_etl_ibis",
            user="postgres",
            password="admin",
            host="localhost",
            port="5432"
        )
        print("Conexión exitosa")
        return conn
    except OperationalError as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None

# Crear la conexión
conn = create_connection()

if conn:
    # Crear un cursor para ejecutar comandos SQL
    cur = conn.cursor()

    # Verificar la conexión
    cur.execute("SELECT version();")
    print(cur.fetchone())

    # Cerrar la conexión
    cur.close()
    conn.close()

Conexión exitosa
('PostgreSQL 17.0 on x86_64-windows, compiled by msvc-19.41.34120, 64-bit',)


## 2. Carga de Datos

### Inserto datos en TABLA CIUDAD

In [17]:
# Reabrir la conexión
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

# Convertir DataFrame a lista de tuplas
records = df_api.to_records(index=False)
records_list = list(records)

# Insertar múltiples registros con ejecuta_many()
query = """
INSERT INTO ciudad (nombre_ciudad) VALUES ('Madrid') RETURNING id_ciudad;
"""
cur.executemany(query, records_list)

# Guardar cambios
conn.commit()

# Cerrar la conexión
cur.close()
conn.close()
print("Datos insertados correctamente")

Datos insertados correctamente


### Cargar datos en TABLA HOTELES

In [ ]:
# Reabrir la conexión
conn = psycopg2.connect(dbname="project_etl_ibis", user="postgres", password="admin", host="localhost", port="5432")
cur = conn.cursor()

# Convertir DataFrame a lista de tuplas
records = df_api.to_records(index=False)
records_list = list(records)

# Insertar múltiples registros con ejecuta_many()
query = """
    INSERT INTO eventos (id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion)
    VALUES (%s, %s, %s, %s, %s)
"""
cur.executemany(query, records_list)

# Guardar cambios
conn.commit()

# Cerrar la conexión
cur.close()
conn.close()
print("Datos insertados correctamente")